# Libraries

In [1]:
# Import libraries
# ================

# for date and time opeations
from datetime import datetime
# for file and folder operations
import os
# for regular expression opeations
import re
# for listing files in a folder
import glob
# for getting web contents
import requests 
# storing and analysing data
import pandas as pd
# for scraping web contents
from bs4 import BeautifulSoup
# regular expression
import re
# for numerical analysis
import numpy as np

# Web Scrapping

In [2]:
# Get data
# ========

# link at which web data recides
link = 'https://www.mohfw.gov.in/'
# get web data
req = requests.get(link)
# parse web data
soup = BeautifulSoup(req.content, "html.parser")

In [3]:
# Find the table
# ==============
# Our target table is the last table in the page

# Get the table head
# Table head may contain the column names, titles, subtitles
thead = soup.find_all('thead')[-1]
# print(thead)

# Get all the rows in table head
# It usually have only one row, which has the column names
head = thead.find_all('tr')
# print(head)

# Get the table tbody
# It contains the contents
tbody = soup.find_all('tbody')[-1]
# print(tbody)

# Get all the rows in table body
# Each row is each state's entry
body = tbody.find_all('tr')
# print(body)

In [4]:
# Get the table contents
# ======================

# Container for header rows / column title
head_rows = []
# Container for table body / contents
body_rows = []

# Loop through the head and append each row to head
for tr in head:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    head_rows.append(row)
# print(head_rows)

# Loop through the body and append each row to body
for tr in body:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    body_rows.append(row)
# print(head_rows)

In [5]:
# Save contents in a dataframe
# ============================
    
# Skip last 3 rows, it contains unwanted info
# head_rows contains column title
df_bs = pd.DataFrame(body_rows[:len(body_rows)-6], 
                     columns=head_rows[0])         

# Drop 'S. No.' column
df_bs.drop('S. No.', axis=1, inplace=True)

# There are 36 states+UT in India
df_bs.head(36)

,Name of State / UT,Active Cases*,Cured/Discharged/Migrated*,Deaths**,Total Confirmed cases*
0,Andaman and Nicobar Islands,47,133,0,180
1,Andhra Pradesh,18159,19393,492,38044
2,Arunachal Pradesh,387,153,3,543
3,Assam,6818,12888,48,19754
4,Bihar,7549,14018,197,21764
5,Chandigarh,164,476,11,651
6,Chhattisgarh,1260,3451,21,4732
7,Dadra and Nagar Haveli and Daman and Diu,179,371,2,552
8,Delhi,17407,97693,3545,118645
9,Goa,1272,1817,19,3108


# Data Cleaning

In [6]:
# Date-Time Information
# =====================

# Today's date
now  = datetime.now()
# Format date to month-day-year
df_bs['Date'] = now.strftime("%m/%d/%Y") 

# Add 'Date' column to dataframe
df_bs['Date'] = pd.to_datetime(df_bs['Date'], format='%m/%d/%Y')

# df_bs.head(36)

In [7]:
# Remove extra characters from 'Name of State/UT' column
df_bs['Name of State / UT'] = df_bs['Name of State / UT'].str.replace('#', '')

In [8]:
# Latitude and longitude information
# ==================================

# Latitude of the states
lat = {'Delhi':28.7041, 'Haryana':29.0588, 'Kerala':10.8505, 'Rajasthan':27.0238,
       'Telengana':18.1124, 'Telangana':18.1124, 'Uttar Pradesh':26.8467, 'Ladakh':34.2996, 'Tamil Nadu':11.1271,
       'Jammu and Kashmir':33.7782, 'Punjab':31.1471, 'Karnataka':15.3173, 'Maharashtra':19.7515,
       'Andhra Pradesh':15.9129, 'Odisha':20.9517, 'Uttarakhand':30.0668, 'West Bengal':22.9868, 
       'Puducherry': 11.9416, 'Chandigarh': 30.7333, 'Chhattisgarh':21.2787, 'Gujarat': 22.2587, 
       'Himachal Pradesh': 31.1048, 'Madhya Pradesh': 22.9734, 'Bihar': 25.0961, 'Manipur':24.6637, 
       'Mizoram':23.1645, 'Goa': 15.2993, 'Andaman and Nicobar Islands': 11.7401, 'Assam' : 26.2006, 
       'Jharkhand': 23.6102, 'Arunachal Pradesh': 28.2180, 'Tripura': 23.9408, 'Nagaland': 26.1584, 
       'Meghalaya' : 25.4670, 'Dadar Nagar Haveli' : 20.1809, 'Sikkim':27.5330, 
       'Dadra and Nagar Haveli and Daman and Diu': 20.1809}

# Longitude of the states
long = {'Delhi':77.1025, 'Haryana':76.0856, 'Kerala':76.2711, 'Rajasthan':74.2179,
        'Telengana':79.0193, 'Telangana':79.0193, 'Uttar Pradesh':80.9462, 'Ladakh':78.2932, 'Tamil Nadu':78.6569,
        'Jammu and Kashmir':76.5762, 'Punjab':75.3412, 'Karnataka':75.7139, 'Maharashtra':75.7139,
        'Andhra Pradesh':79.7400, 'Odisha':85.0985, 'Uttarakhand':79.0193, 'West Bengal':87.8550, 
        'Puducherry': 79.8083, 'Chandigarh': 76.7794, 'Chhattisgarh':81.8661, 'Gujarat': 71.1924, 
        'Himachal Pradesh': 77.1734, 'Madhya Pradesh': 78.6569, 'Bihar': 85.3131, 'Manipur':93.9063, 
        'Mizoram':92.9376, 'Goa': 74.1240, 'Andaman and Nicobar Islands': 92.6586, 'Assam' : 92.9376, 
        'Jharkhand': 85.2799, 'Arunachal Pradesh': 94.7278, 'Tripura': 91.9882, 'Nagaland': 94.5624,
        'Meghalaya' : 91.3662, 'Dadar Nagar Haveli' : 73.0169, 'Sikkim':88.5122,
        'Dadra and Nagar Haveli and Daman and Diu': 73.0169}

# Add latitude column based on 'Name of State / UT' column
df_bs['Latitude'] = df_bs['Name of State / UT'].map(lat)

# Add longitude column based on 'Name of State / UT' column
df_bs['Longitude'] = df_bs['Name of State / UT'].map(long)

# df_bs.head(36)

In [9]:
# Unique state names
df_bs['Name of State / UT'].unique()

array(['Andaman and Nicobar Islands', 'Andhra Pradesh',
       'Arunachal Pradesh', 'Assam', 'Bihar', 'Chandigarh',
       'Chhattisgarh', 'Dadra and Nagar Haveli and Daman and Diu',
       'Delhi', 'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh',
       'Jammu and Kashmir', 'Jharkhand', 'Karnataka', 'Kerala', 'Ladakh',
       'Madhya Pradesh', 'Maharashtra', 'Manipur', 'Meghalaya', 'Mizoram',
       'Nagaland', 'Odisha', 'Puducherry', 'Punjab', 'Rajasthan',
       'Sikkim', 'Tamil Nadu', 'Telangana', 'Tripura', 'Uttarakhand',
       'Uttar Pradesh', 'West Bengal'], dtype=object)

In [10]:
# Number of missing values 
df_bs.isna().sum()

Name of State / UT            0
Active Cases*                 0
Cured/Discharged/Migrated*    0
Deaths**                      0
Total Confirmed cases*        0
Date                          0
Latitude                      0
Longitude                     0
dtype: int64

In [11]:
# Number of unique values 
df_bs.nunique()

Name of State / UT            35
Active Cases*                 35
Cured/Discharged/Migrated*    35
Deaths**                      28
Total Confirmed cases*        35
Date                           1
Latitude                      35
Longitude                     31
dtype: int64

# Saving data

In [12]:
# Saving data
# ===========

# File names as year-month-day.csv format
file_name = now.strftime("%Y_%m_%d")+'.csv'

# Location for saving the file
file_loc = "C:/Users/Abhijeet/Documents/COVID19-Dashboard/data/day_by_day_data_"

# Save file as a scv file
df_bs.to_csv(file_loc + file_name, index=False)

df_bs.head(36)

,Name of State / UT,Active Cases*,Cured/Discharged/Migrated*,Deaths**,Total Confirmed cases*,Date,Latitude,Longitude
0,Andaman and Nicobar Islands,47,133,0,180,2020-07-17,11.7401,92.6586
1,Andhra Pradesh,18159,19393,492,38044,2020-07-17,15.9129,79.7400
2,Arunachal Pradesh,387,153,3,543,2020-07-17,28.2180,94.7278
3,Assam,6818,12888,48,19754,2020-07-17,26.2006,92.9376
4,Bihar,7549,14018,197,21764,2020-07-17,25.0961,85.3131
5,Chandigarh,164,476,11,651,2020-07-17,30.7333,76.7794
6,Chhattisgarh,1260,3451,21,4732,2020-07-17,21.2787,81.8661
7,Dadra and Nagar Haveli and Daman and Diu,179,371,2,552,2020-07-17,20.1809,73.0169
8,Delhi,17407,97693,3545,118645,2020-07-17,28.7041,77.1025
9,Goa,1272,1817,19,3108,2020-07-17,15.2993,74.1240


In [13]:
# column names 
# df_bs.columns

# Combining data

In [14]:
# Location of the file
loc = "C:/Users/Abhijeet/Documents/COVID19-Dashboard/data/day_by_day_data_"

# List of all files
files = glob.glob(loc+'2020*.csv')
   
# Container for each day's data's dataframe
dfs = []

# Loop through the files and append to the dfs list
for i in files:
    # read data
    df_temp = pd.read_csv(i)
    
    # rename columns
    
    try:
        df_temp = df_temp.drop(['Total Confirmed cases (Indian National)', 
                                'Total Confirmed cases ( Foreign National )'], axis=1)
    except:
        pass
        
    d = {'^Cured.*': 'Cured/Discharged/Migrated', 
         'Total Confirmed cases.*': 'Total Confirmed cases', 
         'Death.*': 'Death'}
    
    df_temp.columns = df_temp.columns.to_series().replace(d, regex=True)


#     df_temp = df_temp.rename(columns={'Cured':'Cured/Discharged'})
#     df_temp = df_temp.rename(columns={'Cured/Discharged':'Cured/Discharged/Migrated', 
#                                       'Total Confirmed cases *': 'Total Confirmed cases', 
#                                       'Total Confirmed cases ': 'Total Confirmed cases', 
#                                       'Total Confirmed cases* ': 'Total Confirmed cases'})
#     df_temp = df_temp.rename(columns=lambda x: re.sub('Total Confirmed cases \(Including .* foreign Nationals\) ',
#                                                       'Total Confirmed cases',x))
#     df_temp = df_temp.rename(columns=lambda x: re.sub("Death.*", "Death", x))

    
    # append to the df_s
    dfs.append(df_temp)
    
# print(dfs)

# Concat dataframes
complete_data = pd.concat(dfs, ignore_index=True).sort_values(['Date'], ascending=True).reset_index(drop=True)

# Get just numbers
complete_data['Death'] = complete_data['Death'].astype('str').str.extract('(\d+)').astype('int')

# Few sample rows
complete_data[10:15]

,Name of State / UT,Active Cases*,Cured/Discharged/Migrated,Death,Total Confirmed cases,Date,Latitude,Longitude
10,Sikkim,155,88,0,243,2020-07-17,27.5330,88.5122
11,Tamil Nadu,46717,107416,2236,156369,2020-07-17,11.1271,78.6569
12,Telangana,13327,27295,396,41018,2020-07-17,18.1124,79.0193
13,Tripura,676,1604,3,2283,2020-07-17,23.9408,91.9882
14,Uttarakhand,937,2995,50,3982,2020-07-17,30.0668,79.0193


In [15]:
# complete_data.columns

## Preprocessing

In [16]:
# Fix datatype
complete_data['Date'] = pd.to_datetime(complete_data['Date'])

# Sort rows
complete_data = complete_data.sort_values(['Date', 'Name of State / UT']).reset_index(drop=True)

# Fill missing values with 0
cols = ['Cured/Discharged/Migrated', 'Death']
complete_data[cols] = complete_data[cols].fillna(0).astype('int')

In [17]:
# Rename State/UT names
complete_data['Name of State / UT'].replace('Chattisgarh', 'Chhattisgarh', inplace=True)
complete_data['Name of State / UT'].replace('Pondicherry', 'Puducherry', inplace=True) 

In [18]:
# Select only rows with more than 1 case
complete_data = complete_data[complete_data['Total Confirmed cases']>0]

In [19]:
# drop extra columns
# complete_data = complete_data.drop(['Active Cases*'], axis=1)

In [20]:
complete_data

,Name of State / UT,Active Cases*,Cured/Discharged/Migrated,Death,Total Confirmed cases,Date,Latitude,Longitude
0,Andaman and Nicobar Islands,47,133,0,180,2020-07-17,11.7401,92.6586
1,Andhra Pradesh,18159,19393,492,38044,2020-07-17,15.9129,79.7400
2,Arunachal Pradesh,387,153,3,543,2020-07-17,28.2180,94.7278
3,Assam,6818,12888,48,19754,2020-07-17,26.2006,92.9376
4,Bihar,7549,14018,197,21764,2020-07-17,25.0961,85.3131
5,Chandigarh,164,476,11,651,2020-07-17,30.7333,76.7794
6,Chhattisgarh,1260,3451,21,4732,2020-07-17,21.2787,81.8661
7,Dadra and Nagar Haveli and Daman and Diu,179,371,2,552,2020-07-17,20.1809,73.0169
8,Delhi,17407,97693,3545,118645,2020-07-17,28.7041,77.1025
9,Goa,1272,1817,19,3108,2020-07-17,15.2993,74.1240


In [21]:
# Rearrange columns
complete_data = complete_data[['Date', 'Name of State / UT', 'Active Cases*', 'Latitude', 'Longitude', 
                               'Total Confirmed cases', 'Death', 'Cured/Discharged/Migrated']]

In [22]:
# New cases
# =========

# Temp dataset
temp = complete_data.groupby(['Name of State / UT', 'Date', ])['Total Confirmed cases', 'Death', 'Cured/Discharged/Migrated']
temp = temp.sum().diff().reset_index()

mask = temp['Name of State / UT'] != temp['Name of State / UT'].shift(1)

temp.loc[mask, 'Total Confirmed cases'] = np.nan
temp.loc[mask, 'Death'] = np.nan
temp.loc[mask, 'Cured/Discharged/Migrated'] = np.nan

temp = temp[['Date', 'Name of State / UT', 'Total Confirmed cases', 'Death', 'Cured/Discharged/Migrated']]
temp.columns = ['Date', 'Name of State / UT', 'New cases', 'New deaths', 'New recovered']

# Merging new values
complete_data = pd.merge(complete_data, temp, on=['Name of State / UT', 'Date'])

# Filling na with 0
complete_data = complete_data.fillna(0)

# Fixing data types
cols = ['New cases', 'New deaths', 'New recovered']
complete_data[cols] = complete_data[cols].astype('int')

# Remove negative values
complete_data['New cases'] = complete_data['New cases'].apply(lambda x: 0 if x<0 else x)

# Final data
complete_data

C:\Users\Abhijeet\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """


,Date,Name of State / UT,Active Cases*,Latitude,Longitude,Total Confirmed cases,Death,Cured/Discharged/Migrated,New cases,New deaths,New recovered
0,2020-07-17,Andaman and Nicobar Islands,47,11.7401,92.6586,180,0,133,0,0,0
1,2020-07-17,Andhra Pradesh,18159,15.9129,79.7400,38044,492,19393,0,0,0
2,2020-07-17,Arunachal Pradesh,387,28.2180,94.7278,543,3,153,0,0,0
3,2020-07-17,Assam,6818,26.2006,92.9376,19754,48,12888,0,0,0
4,2020-07-17,Bihar,7549,25.0961,85.3131,21764,197,14018,0,0,0
5,2020-07-17,Chandigarh,164,30.7333,76.7794,651,11,476,0,0,0
6,2020-07-17,Chhattisgarh,1260,21.2787,81.8661,4732,21,3451,0,0,0
7,2020-07-17,Dadra and Nagar Haveli and Daman and Diu,179,20.1809,73.0169,552,2,371,0,0,0
8,2020-07-17,Delhi,17407,28.7041,77.1025,118645,3545,97693,0,0,0
9,2020-07-17,Goa,1272,15.2993,74.1240,3108,19,1817,0,0,0


In [23]:
complete_data['Name of State / UT'] = complete_data['Name of State / UT'].replace('Dadar Nagar Haveli', 'Dadra and Nagar Haveli and Daman and Diu')
complete_data['Name of State / UT'] = complete_data['Name of State / UT'].replace('Daman & Diu', 'Dadra and Nagar Haveli and Daman and Diu')

In [24]:
sorted(complete_data['Name of State / UT'].unique())

['Andaman and Nicobar Islands',
 'Andhra Pradesh',
 'Arunachal Pradesh',
 'Assam',
 'Bihar',
 'Chandigarh',
 'Chhattisgarh',
 'Dadra and Nagar Haveli and Daman and Diu',
 'Delhi',
 'Goa',
 'Gujarat',
 'Haryana',
 'Himachal Pradesh',
 'Jammu and Kashmir',
 'Jharkhand',
 'Karnataka',
 'Kerala',
 'Ladakh',
 'Madhya Pradesh',
 'Maharashtra',
 'Manipur',
 'Meghalaya',
 'Mizoram',
 'Nagaland',
 'Odisha',
 'Puducherry',
 'Punjab',
 'Rajasthan',
 'Sikkim',
 'Tamil Nadu',
 'Telangana',
 'Tripura',
 'Uttar Pradesh',
 'Uttarakhand',
 'West Bengal']

## Final dataframe

In [25]:
# Random rows
complete_data.sample(3)

,Date,Name of State / UT,Active Cases*,Latitude,Longitude,Total Confirmed cases,Death,Cured/Discharged/Migrated,New cases,New deaths,New recovered
14,2020-07-17,Jharkhand,2069,23.6102,85.2799,4624,42,2513,0,0,0
9,2020-07-17,Goa,1272,15.2993,74.1240,3108,19,1817,0,0,0
15,2020-07-17,Karnataka,30661,15.3173,75.7139,51422,1032,19729,0,0,0


In [26]:
# Complete data info
complete_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35 entries, 0 to 34
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Date                       35 non-null     datetime64[ns]
 1   Name of State / UT         35 non-null     object        
 2   Active Cases*              35 non-null     int64         
 3   Latitude                   35 non-null     float64       
 4   Longitude                  35 non-null     float64       
 5   Total Confirmed cases      35 non-null     int64         
 6   Death                      35 non-null     int32         
 7   Cured/Discharged/Migrated  35 non-null     int32         
 8   New cases                  35 non-null     int64         
 9   New deaths                 35 non-null     int32         
 10  New recovered              35 non-null     int32         
dtypes: datetime64[ns](1), float64(2), int32(4), int64(3), object(1)
memory us

## Save as .csv file

In [27]:
# Save data in a csv file
complete_data.to_csv('data/complete.csv', index=False)

In [28]:
complete_data.groupby('Date').count()

,Name of State / UT,Active Cases*,Latitude,Longitude,Total Confirmed cases,Death,Cured/Discharged/Migrated,New cases,New deaths,New recovered
Date,,,,,,,,,,
2020-07-17,35,35,35,35,35,35,35,35,35,35


In [29]:
complete_data.sort_values('Death', ascending=False)

,Date,Name of State / UT,Active Cases*,Latitude,Longitude,Total Confirmed cases,Death,Cured/Discharged/Migrated,New cases,New deaths,New recovered
19,2020-07-17,Maharashtra,114947,19.7515,75.7139,284281,11194,158140,0,0,0
8,2020-07-17,Delhi,17407,28.7041,77.1025,118645,3545,97693,0,0,0
29,2020-07-17,Tamil Nadu,46717,11.1271,78.6569,156369,2236,107416,0,0,0
10,2020-07-17,Gujarat,11289,22.2587,71.1924,45481,2089,32103,0,0,0
32,2020-07-17,Uttar Pradesh,15720,26.8467,80.9462,43441,1046,26675,0,0,0
15,2020-07-17,Karnataka,30661,15.3173,75.7139,51422,1032,19729,0,0,0
34,2020-07-17,West Bengal,13679,22.9868,87.8550,36117,1023,21415,0,0,0
18,2020-07-17,Madhya Pradesh,5562,22.9734,78.6569,20378,689,14127,0,0,0
27,2020-07-17,Rajasthan,6666,27.0238,74.2179,27174,538,19970,0,0,0
1,2020-07-17,Andhra Pradesh,18159,15.9129,79.7400,38044,492,19393,0,0,0


In [30]:
df = complete_data[complete_data['Date']==max(complete_data['Date'])]

In [31]:
df = df.to_csv('data/latest.csv')

In [32]:
df2 = pd.read_csv('data/latest.csv')
df2.head()

,Unnamed: 0,Date,Name of State / UT,Active Cases*,Latitude,Longitude,Total Confirmed cases,Death,Cured/Discharged/Migrated,New cases,New deaths,New recovered
0,0,2020-07-17,Andaman and Nicobar Islands,47,11.7401,92.6586,180,0,133,0,0,0
1,1,2020-07-17,Andhra Pradesh,18159,15.9129,79.7400,38044,492,19393,0,0,0
2,2,2020-07-17,Arunachal Pradesh,387,28.2180,94.7278,543,3,153,0,0,0
3,3,2020-07-17,Assam,6818,26.2006,92.9376,19754,48,12888,0,0,0
4,4,2020-07-17,Bihar,7549,25.0961,85.3131,21764,197,14018,0,0,0


In [33]:
df2 = df2.drop(['Unnamed: 0', 'Latitude', 'Longitude'], axis=1)

In [34]:
df2.to_csv('data/final.csv', index=False)

In [35]:
df2

,Date,Name of State / UT,Active Cases*,Total Confirmed cases,Death,Cured/Discharged/Migrated,New cases,New deaths,New recovered
0,2020-07-17,Andaman and Nicobar Islands,47,180,0,133,0,0,0
1,2020-07-17,Andhra Pradesh,18159,38044,492,19393,0,0,0
2,2020-07-17,Arunachal Pradesh,387,543,3,153,0,0,0
3,2020-07-17,Assam,6818,19754,48,12888,0,0,0
4,2020-07-17,Bihar,7549,21764,197,14018,0,0,0
5,2020-07-17,Chandigarh,164,651,11,476,0,0,0
6,2020-07-17,Chhattisgarh,1260,4732,21,3451,0,0,0
7,2020-07-17,Dadra and Nagar Haveli and Daman and Diu,179,552,2,371,0,0,0
8,2020-07-17,Delhi,17407,118645,3545,97693,0,0,0
9,2020-07-17,Goa,1272,3108,19,1817,0,0,0


In [36]:
import pandas as pd
df3 = pd.read_csv("data/final.csv")
df3.head()

,Date,Name of State / UT,Active Cases*,Total Confirmed cases,Death,Cured/Discharged/Migrated,New cases,New deaths,New recovered
0,2020-07-17,Andaman and Nicobar Islands,47,180,0,133,0,0,0
1,2020-07-17,Andhra Pradesh,18159,38044,492,19393,0,0,0
2,2020-07-17,Arunachal Pradesh,387,543,3,153,0,0,0
3,2020-07-17,Assam,6818,19754,48,12888,0,0,0
4,2020-07-17,Bihar,7549,21764,197,14018,0,0,0
